# Grid Loop
## Author:  Dominic Doud (NASA Ames Research Center)

You should have already installed picaso (which can be found [here](https://natashabatalha.github.io/picaso/installation.html)) and ran the previous notebook [AnalyzeExoplanet_picaso_1_setup.ipynb](https://colab.research.google.com/drive/1zPjFDy8HTy4nwsFbAbq73UJMZkTMZGCT?usp=sharing) and to download the Wasp 39b data, and walked through the second part [AnalyzeExoplanet_2_picaso.ipynb](www.google.com). 

You should know how to output one model with an associated adding/removing of certain chemicals from [AnalyzeExoplanet_2_picaso](www.google.com). It is important to understand that a proper analysis requires more than one model in order to succeed in the most accuracy in an analysis, such as seen in the [AnalyzeExoplanet_4_Ultranest](www.google.com) notebook. However, instead of spending hours or days creating hundreds of models, you will be downloading a grid in the Ultranest tutorial. However, it is important to understand how to make that amount of models in order to run your own analysis of your own planet. 

The main goal you want to achieve is to be able to create multiple models that vary multiple variables including but not limited to:
- CtoO ratio
- Metallicity
- Internal Temperature
- rfacv

which is what the code does here. In some cases you may need more variables, some cases you may need less, the importance is not to overfit or have to little constraint. This is not necessarily the correct or maybe not even the most efficient way, but this is what I used and will create the files needed for a typical Ultranest analysis (although I would make way more than 8 or so models, but understanding the process is what is key here). I cannot tell you what these values should be, as they change depending on your planet and star system. However, whatever files you need, you'll need the corresponding k table. These can be downloaded [here](https://zenodo.org/records/7542068) (make sure to extract!) At the end of this grid loop, you should have two folders `.../xarrays/climate/` and `.../xarrays/hi_res/`. These should be in `.nc` format, or an xarray file. This is the only way that it will work with grid fitting via Ultranest. 

In this case, I used:

- sonora_2020_feh+000_co_100_noTiOVO.data.196.tar.gz
- sonora_2020_feh+000_co_250_noTiOVO.data.196.tar.gz
- sonora_2020_feh+100_co_100_noTiOVO.data.196.tar.gz
- sonora_2020_feh+100_co_250_noTiOVO.data.196.tar.gz

This could take from ~15-30 minutes.

In [12]:
def run_climate(base_case_name, array_save_dir, mh, CtoO, t_int, star_temp, star_metal, star_radius, star_logg, semi_major, nlevel, nstr_upper,
                rfacv, opacity_ph, teq, planet_mass=0, planet_radius=0, planet_gravity=0,
                resume=False, nofczns=1):
    #check if done already and want to skip:
    #create file name
    print("Running climate")
    savefile=base_case_name+"_tint"+str(t_int)+"_rfacv"+str(rfacv)+"_mh"+mh+"_cto"+CtoO+".nc"
    if (resume & os.path.exists(os.path.join(array_save_dir,'climate',savefile))): 
        print('Skipping',savefile,', file already exists')
        return
    else:
        #create case/inputs    
        case_name = jdi.inputs(calculation="planet", climate = True)
        case_name.star(opacity_ph, temp=star_temp,metal=star_metal,logg=star_logg,
                        radius=star_radius, semi_major= semi_major ,
                        database='phoenix', radius_unit = jdi.u.Unit('R_sun'), semi_major_unit = jdi.u.AU)
        case_name.gravity(mass=planet_mass, mass_unit=jdi.u.Unit('M_jup'), 
                      radius=planet_radius, radius_unit=jdi.u.Unit('R_jup'))
        #calc teq and check params(uncomment if needed)
        #print("Teq: " + str(teq))
        #print(t_int)
        #print("mh: " + mh + "\nCtoO: " + CtoO + "\nT_int: " + str(t_int) + "\nStar_temp: " + str(star_temp) + "\nStar_metal: " + str(star_metal)
        #     + "\nStar_radius: " + str(star_radius) + "\nStar_logg: " + str(star_logg) + "\nSemi_major: " + str(semi_major) + "\nnlevel: " + str(nlevel)
        #    + "\nnstr_uppter: " + str(nstr_upper) + "\nrfacv: " + str(rfacv) + "\nplanet_mass: " + str(planet_mass) + "\nplanet_radius: " + str(planet_radius)
        #     + "\nplanet_grav: " + str(planet_gravity) + "\nResume: " + str(resume) + "\nnofczns: " + str(nofczns))
        case_name.effective_temp(t_int)
        pt = case_name.guillot_pt(teq, nlevel=nlevel, T_int = t_int, p_bottom=2, p_top=-6) 
        temp_guess = pt['temperature'].values 
        pressure = pt['pressure'].values
        #print(temp_guess, pressure)
    
        #time to find convection zone!
    
        nstr_deep = nlevel -2 # this is always the case
        nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones
        #print('get climate')
        
        case_name.inputs_climate(temp_guess=temp_guess, pressure=pressure, 
                          nstr=nstr, nofczns=nofczns, rfacv=rfacv)
        #print('inputs_climate')
        #print(opacity_ph)
        #create climate, atmosphere from climate, spectrum from climate modeling and export to xarr
        out = case_name.climate(opacity_ph, save_all_profiles=True, with_spec=False) 
        #print('create climate')
        #create atmosphere from climate
        case_name.atmosphere(out['ptchem_df'])
    
        #create spectrum from climate modeling and export to xarr
        df = case_name.spectrum(opacity_ph,calculation='transmission+thermal', full_output=True)
        
        #make folder if needed
        if not os.path.exists(os.path.join(array_save_dir,'climate')):
            os.mkdir(os.path.join(array_save_dir,'climate'))
        print("Saving xarray: " + savefile)
        jdi.output_xarray(df, case_name, add_output={'author': 'Dominic Doud'}, savefile=os.path.join(array_save_dir,"climate",savefile))
        return out

def hi_res_spec_xarr(out, array_save_dir, R, base_case_name, mh, CtoO, t_int, star_temp, star_metal, star_logg, star_radius, semi_major, rfacv, resume=False, planet_gravity=0, planet_radius=0, planet_mass=0):
    #check if done already and want to skip:
    #create file name
    print("Running hi-res spec")
    savefile="hi_res_"+base_case_name+"_tint"+str(t_int)+"_rfacv"+str(rfacv)+"_mh"+mh+"_cto"+CtoO+".nc"
    if (resume & os.path.exists(os.path.join(array_save_dir,'hi_res',savefile))): 
        print('Skipping',savefile,', file already exists')
        return
    else:
        hi_res = jdi.inputs(calculation="planet")
        hi_res.gravity(mass=planet_mass, mass_unit=jdi.u.Unit('M_jup'), radius=planet_radius, radius_unit=jdi.u.Unit('R_jup')) # input gravity
        hi_res.star(opacity_full, temp=star_temp,metal=star_metal,logg=star_logg,radius=star_radius, semi_major= semi_major , semi_major_unit = jdi.u.AU, database='phoenix', radius_unit = jdi.u.Unit('R_sun') )
    
        hi_res.atmosphere(df=out['ptchem_df'])
        df_spec = hi_res.spectrum(opacity_full, full_output=True, calculation='thermal')
        w,f = jdi.mean_regrid(df_spec['wavenumber'],df_spec['thermal'], R=R)
        
        #make folder if needed
        if not os.path.exists(os.path.join(array_save_dir,'hi_res')):
            os.mkdir(os.path.join(array_save_dir,'hi_res'))
        print("Saving xarray: " + savefile)
        jdi.output_xarray(df_spec, hi_res, add_output={'author': 'Dominic Doud'}, savefile=os.path.join(array_save_dir,"hi_res",savefile))
        #jpi.show(jpi.spectrum(w,f, y_axis_label='Thermal Flux'))

In [10]:
import os
#check you have picaso
from picaso import justdoit as jdi
from picaso import justplotit as jpi
import picaso.opacity_factory as op
import numpy as np

jpi.output_notebook()
# define the same directory you did in the setup, where you have extracted the example data files
picaso_tutorial_dir = 'exoplanet_modeling_tutorial'

Loading BokehJS ...

In [1]:
correlated_k_dir ='exoplanet_modeling_tutorial/correlated_k_tables/' #direct to wherever you downloaded the correlated_k_tables
array_save_dir ='exoplanet_modeling_tutorial/xarrays/'
mh_list = ['+000', '+100']
CtoO_list = ['100', '250']
rfacv_list=[0.5]
t_int_list=[200, 300]
#continue where left off?
resume=False
#other variables
base_case_name='wasp-39b'
R=100 #resolution power
#wav range
min_wav=3
max_wav=15

nstr_upper = 85
nlevel = 91 # starting with min amount of pressure levels (51-91)

#planet/star params
star_temp = 5400
star_metal = 0.01
star_radius = 0.9
star_logg = 4.45

wasp_mass=0.28
wasp_radius=1.27
teq=1120.55
semi_major=0.0486


#run the loops!
opacity_full = jdi.opannection(wave_range=[min_wav,max_wav])
for mh in mh_list:
    for CtoO in CtoO_list:
        for rfacv in rfacv_list:
            for t_int in t_int_list:
                #grab opacities
                print(mh, CtoO)
                ph_db = correlated_k_dir+f'sonora_2020_feh{mh}_co_{CtoO}_noTiOVO.data.196'
                print(ph_db)
                opacity_ph = jdi.opannection(ck_db=ph_db)
                #get climate xarray
                first_out = run_climate(base_case_name, array_save_dir, mh, CtoO, t_int, star_temp, star_metal,
                            star_radius, star_logg, semi_major, nlevel, nstr_upper, rfacv, opacity_ph, teq=teq,
                            planet_mass=wasp_mass, planet_radius=wasp_radius, resume=resume)
                #get hi res xarray
                hi_res_spec_xarr(first_out, array_save_dir, R, base_case_name, mh, CtoO, t_int, 
                                               star_temp, star_metal, star_logg, 
                                               star_radius, semi_major, rfacv, resume, planet_radius=wasp_radius, 
                                               planet_mass=wasp_mass)

NameError: name 'jdi' is not defined